Reaction Diffusion Models
=========================

**Author:** Ulrich G. Wortmann



\maketitle



## Preparing your session



### Installing the python modules needed in this session



You need to do this once per session 



In [1]:
%pip install fastfd

### Import the model and the plotting interface



In [1]:
import matplotlib.pyplot as plt
import pandas as pd
from diff_lib import data_container 
from run_methane import model

## Defining the first model



Model base units are in meter and seconds. Concentrations are in mmol/l.



In [1]:
# a few parameters to play with
p = {
    "max_depth": 0.35,  # meters
    "grid_points": 300,
    "phi": 0.65,  # porosity
    "w": 1.5e-10,  # sedimentation rate in m/s
     
}

### Set boundary conditions



In [1]:
bc = {
    "ch4": [  # species
        "concentration",  # upper bc type
        0,  # upper bc value
        "concentration",  # lower bc type
        0.3,  # lower bc value
        "dissolved",  # phase
        1,  # reaction type 1 = source, -1 = sink
    ],
}

### Define reaction rates and a reaction term



For this first run, we set all reaction terms to zero. Even so we only consider a single species, the model code requires that we set a reaction constant for each species.



In [1]:
# --------------------------- Reaction constants ------------ *
k = data_container({"ch4": 0})

Furthermore, we need to provide a function that describes who reacts with whom and how. In this case, this function simply returns zero.



In [1]:
def diagenetic_reactions(a, z, c, k, f):
        """Define microbial reactions. Note that reactions
        are always positive. The sign is set by reaction type
        which can be either a 'source' or a 'sink'
        """
        f.ch4 = 0
        
        return f, 0

### Run the model and plot the results



We call  the model code with our parameter list `p`, the boundary conditions `bc`, the reaction rates `k` and our function that describes the diagenetic reactions. The model than returns a dataframe with all the results.



In [1]:
df = model(p, bc, k, diagenetic_reactions)
display(df.head())

### Plot the results



In [1]:
fig, ax = plt.subplots() # Create a new plot object 
ax.plot(df.c_ch4, df.z) # assign data to X & Y axes 
ax.set_xlabel(r"CH$_4$ [mmol/l]") # set labels 
ax.set_ylabel("Depth [mbsf]")
ax.invert_yaxis() 
fig.tight_layout()
plt.show()

### Now what does this all mean?



On first sight, the results seems trivial, however, there some dragon here.

